In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from model.base_model import base_model
import transformers
from transformers import BertConfig, AutoTokenizer
import math
import os
from dataclasses import dataclass
from typing import List, Optional, Tuple, Union
from model.base_model import AdapterModule
import torch
import torch.utils.checkpoint
from torch import nn
from transformers import CLIPTextModel
from peft import PeftConfig, PeftModel

from transformers.activations import ACT2FN
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.pytorch_utils import apply_chunking_to_forward, find_pruneable_heads_and_indices, prune_linear_layer
from transformers.utils import (
    ModelOutput,
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
)
from transformers.models.bert.configuration_bert import BertConfig
# from peft import LoraConfig, TaskType
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType, PeftModel, PeftConfig # add
from transformers import BertModel, BertTokenizer
from adapters import LoRAConfig, BertAdapterModel, PromptTuningConfig, IA3Config
import adapters.composition as ac


config_prompt = PromptTuningConfig(prompt_length=50)
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = IA3Config()

from adapters import ConfigUnion, PrefixTuningConfig, SeqBnConfig
from peft import LoraConfig, get_peft_model, PeftModel


lora_config = LoraConfig(
    r=64, 
    lora_alpha=128, 
    target_modules=["query", "value"],
    lora_dropout=0.05, 
    # use_gating=True
)

config = PeftConfig.from_pretrained('/mnt/ngoclt/Continual Learning/SCKD_ori/SCKD/Checkpoint/LoRA')

# model = get_peft_model(model, configs)

# model = PeftModel.from_pretrained(model, configs, adapter_name="luungoc")

# model.load_adapter(configs, adapter_name="ngocluu")

# model.add_adapter("lora_adapter", config=configs)

# model.set_active_adapters("lora_adapter")

# model.train_adapter("lora_adapter")

print(f"Param trainable: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Param trainable: 109482240


In [26]:
samples = []
with open('/home/luungoc/Continual Learning/ConPL/data/tacred/CFRLdata_10_100_10_5/train_0.txt') as file_in:
    for line in file_in:
        items = line.strip().split('\t')
        if (len(items[0]) > 0):
            relation_ix = int(items[0])
            if items[1] != 'noNegativeAnswer':
                candidate_ixs = [int(ix) for ix in items[1].split()]
                sentence = items[2].split('\n')[0]
                headent = items[3]
                headidx = [int(ix) for ix in items[4].split()]
                tailent = items[5]
                tailidx = [int(ix) for ix in items[6].split()]
                headid = items[7]
                tailid = items[8]
                samples.append(
                    [relation_ix, candidate_ixs, sentence, headent, headidx, tailent, tailidx,
                        headid, tailid])

In [28]:
len(samples)

775

In [44]:
samples = []
with open('/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/train_0.txt') as file_in:
    for line in file_in:
        items = line.strip().split('\t')
        if (len(items[0]) > 0):
            relation_ix = int(items[0])
            if items[1] != 'noNegativeAnswer':
                candidate_ixs = [int(ix) for ix in items[1].split()]
                sentence = items[2].split('\n')[0]
                headent = items[3]
                headidx = [int(ix) for ix in items[4].split()]
                tailent = items[5]
                tailidx = [int(ix) for ix in items[6].split()]
                headid = items[7]
                tailid = items[8]
                samples.append(
                    [relation_ix, candidate_ixs, sentence, headent, headidx, tailent, tailidx,
                        headid, tailid])

In [36]:
rel_index = np.load('/home/luungoc/Continual Learning/data/tacred/rel_index.npy')
rel_cluster_label = np.load('/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_10/rel_cluster_label_0.npy')

In [37]:
print(f'Relation Index: {rel_index}')

print(f'Relation Cluster Label: {rel_cluster_label}')

Relation Index: [ 1  2  3 26  5 29  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25  4 27 28  6 30 31 32 33 34 35 36 37 38 39 40 41]
Relation Cluster Label: [4 1 6 3 2 4 7 1 3 4 3 1 6 0 0 6 5 2 0 7 0 5 4 5 7 1 7 2 5 7 0 6 7 2 3 2 3
 1 6 5 4]


In [38]:
cluster_to_labels = {}
for index, i in enumerate(rel_index):
    if rel_cluster_label[index] in cluster_to_labels.keys():
        cluster_to_labels[rel_cluster_label[index]].append(i-1)
    else:
        cluster_to_labels[rel_cluster_label[index]] = [i-1]

In [39]:
cluster_to_labels

{4: [0, 28, 9, 22, 40],
 1: [1, 7, 11, 3, 37],
 6: [2, 12, 15, 31, 38],
 3: [25, 8, 10, 34, 36],
 2: [4, 17, 27, 33, 35],
 7: [6, 19, 24, 26, 29, 32],
 0: [13, 14, 18, 20, 30],
 5: [16, 21, 23, 5, 39]}